In [1]:
import yfinance as yf
import pandas as pd
import numpy
import math
import altair as alt
import time
import json
import datetime
import sqlite3
import multiprocessing  
import random
from multiprocessing import Pool
import defs
import time
import datetime
import requests
import aiohttp
import asyncio

import re

import requests_cache
#requests_cache.install_cache(cache_name="cache_cache", backend='sqlite')
requests_cache.uninstall_cache()

def Logging(val):
    with open("log.txt") as file: # Use file to refer to the file object
        file.write(val)


#missing777
#fees
#dividends
#stocksplits - what happens in  a stock split(are old shares amount converted to new share amount)
#confirm values, see if value is between open and close of that day
#drop columns when amount is 0

def return_print(msg="",payload=None,status="INFO"):
    
    payload={}
    if payload is None:
        payload=payload

    payload['status']=status
    payload['msg']=msg
    
    print(json.dumps(payload))




#TODO
#fees 
#dividends

In [4]:
#cross results from summaries with results from download
filename = 'Finance_Positions.csv'

f = open(filename, "r")

count_comma=0
count_semicolon=0
for i in range(2):
    line = f.readline()
    count_comma=count_comma+line.count(",")
    count_semicolon=count_semicolon+line.count(";")

f.close()

print("Importing Transactions")

sep=","
if(count_semicolon>count_comma):
    sep=";"

#transactions list
transactions = pd.read_csv(filename,sep=sep)

#get the most earliest date
df=pd.DataFrame()
df['Date']=transactions['When']

#the start date
startdate=pd.to_datetime(df['Date']).sort_values()[0].strftime("%Y-%m-%d")

#get all use tickers distinct
stonks=pd.unique(transactions['Ticker']).tolist()

print("Transactions Loaded")



Importing Transactions
Transactions Loaded


In [9]:
transactions['Total Cost']=transactions['Total Cost'].abs
transactions['Amount']=transactions['Amount'].abs

In [4]:
stonk_currencies
stonk_currencies_missing

[]

In [5]:
#fetches information about the stock (namely currency)

return_print("Fetching Information on the stocks, this may take a minute...")

#fetch from db, if not exist, fetch from internet
stonks_info={}

#get current format
curtime=datetime.datetime.now().isoformat()

inf=0
#serial
for i in stonks:
    try:

    inf=defs.FetchTickerInfo(i)
    except Exception as e:
        print(e)
    stonks_info[i]=inf

    print(i)
    



NameError: name 'stonk_summaries_missing' is not defined

In [6]:
#check if in bd
startdate

'2020-07-08'

In [7]:

#donwload all data
data = yf.download(tickers=stonks,start=startdate,interval='1d',group_by='column',prepos=False,threads=True)


#counts entries without values
empty_stocks  =data['Close'].columns[data['Close'].count()==0].to_list()

if len(empty_stocks)>0:
    payload= {}
    payload['stocks'] = empty_stocks
    return_print("The Following Tickers are Invalid:"+" ".join(empty_stocks)+"\n, tickers must be set accordingly to  https://finance.yahoo.com/",payload,status='ERROR')

return_print("Stock Values Loaded")


[*********************100%***********************]  36 of 36 completed
{"status": "INFO", "msg": "Stock Values Loaded"}


In [8]:
data

Adj Close                                                  \
                  AAL        AAPL        AMD         AMZN        BABA   
Date                                                                    
2020-07-07  11.910000   92.706093  52.930000  3000.120117  236.509995   
2020-07-08  11.990000   94.865234  53.430000  3081.110107  257.679993   
2020-07-09  11.180000   95.273193  57.259998  3182.629883  261.579987   
2020-07-10  11.940000   95.439842  55.880001  3200.000000  261.010010   
2020-07-11        NaN         NaN        NaN          NaN         NaN   
...               ...         ...        ...          ...         ...   
2021-04-13  22.559999  134.429993  80.190002  3400.000000  241.889999   
2021-04-14  22.370001  132.029999  78.550003  3333.000000  239.229996   
2021-04-15  22.129999  134.500000  83.010002  3379.090088  239.089996   
2021-04-16  22.030001  134.160004  82.150002  3399.439941  238.690002   
2021-04-17        NaN         NaN        NaN          NaN         NaN   

                                                                         ...  \
                 BRK-B       BTC-USD         CRM       DIOD         DIS  ...   
Date                                                                     ...   
2020-07-07  181.149994           NaN  196.380005  49.299999  113.629997  ...   
2020-07-08  181.199997   9428.333008  200.279999  49.310001  116.660004  ...   
2020-07-09  178.800003   9277.967773  200.740005  49.790001  116.809998  ...   
2020-07-10  182.899994   9278.807617  198.880005  50.570000  119.339996  ...   
2020-07-11         NaN   9240.346680         NaN        NaN         NaN  ...   
...                ...           ...         ...        ...         ...  ...   
2021-04-13  267.100006  63503.457031  232.000000  78.949997  185.490005  ...   
2021-04-14  267.529999  63109.695312  228.020004  78.989998  187.460007  ...   
2021-04-15  269.140015  63314.011719  233.000000  80.769997  185.929993  ...   
2021-04-16  272.109985  61572.789062  231.910004  81.360001  187.259995  ...   
2021-04-17         NaN  62202.515625         NaN        NaN         NaN  ...   

               Volume                                                   \
                   PG         PLTR           T       TEAM         TSLA   
Date                                                                     
2020-07-07  6866300.0          NaN  27509200.0  1116500.0  107448500.0   
2020-07-08  6494100.0          NaN  38933000.0  1570800.0   81556500.0   
2020-07-09  5341000.0          NaN  39365700.0  1970800.0   58588000.0   
2020-07-10  6752600.0          NaN  36893100.0  1008800.0  116688000.0   
2020-07-11        NaN          NaN         NaN        NaN          NaN   
...               ...          ...         ...        ...          ...   
2021-04-13  9161200.0   83913500.0  41385400.0  2085300.0   44652800.0   
2021-04-14  7584000.0  108793800.0  31287900.0  2079100.0   49017400.0   
2021-04-15  7945300.0   58849300.0  26490800.0  1149800.0   27848900.0   
2021-04-16  7706400.0   46289700.0  30140500.0   900800.0   27924000.0   
2021-04-17        NaN          NaN         NaN        NaN          NaN   

                                                                     
                    V        WDC         WMT         XOM         ZM  
Date                                                                 
2020-07-07  7660100.0  4684600.0  31152700.0  17407600.0  7819700.0  
2020-07-08  8368000.0  3724700.0  17405700.0  18291400.0  6012900.0  
2020-07-09  8548200.0  3716200.0  14257200.0  25882000.0  5396000.0  
2020-07-10  7674200.0  3259800.0  14745600.0  18951000.0  6354800.0  
2020-07-11        NaN        NaN         NaN         NaN        NaN  
...               ...        ...         ...         ...        ...  
2021-04-13  4764000.0  3592700.0   5799300.0  19370600.0  5298200.0  
2021-04-14  4972600.0  2974000.0   7308200.0  28051100.0  2844900.0  
2021-04-15  8698100.0  3444000.0   7236800.0  23250300.0  336

In [9]:
#analyse which pairs to take and downlaod
allcurrencies=[]

for ticker in stonk_currencies:
    allcurrencies.append(stonk_currencies[ticker])
    

curs = set(allcurrencies).difference(('USD',))
print(curs)

curpairs=[]

#generate strings
for i in curs:
    curpairs.append(i+"USD=X")


#download required pairs
pairs = yf.download(tickers=curpairs,start=startdate,interval='1d',group_by='column',prepos=False,threads=True)


{'EUR'}
[*********************100%***********************]  1 of 1 completed


In [10]:
#generate analytics cube

stonkdict={}


targetcurrency='USD'

superdf =pd.DataFrame()
print(superdf.empty)

i=0


for stonk in stonks:  
    
    print(stonk)
    
    #get transactions from csv
    singlestock_transactions = transactions[transactions['Ticker']==stonk]

    #use index to create df
    index = data.index
    df=pd.DataFrame(index=data.index)

    
    #extract price
    df['Price']= data['Close'][stonk]
    #initialize amount
    df["SharesOwned"]=0
    #initialize total value
    df['TotalValue']=0   

    df['InvestedValue']=0   

    #set ticker
    df["Ticker"]=stonk
    
    #fetch currency
    df["Currency"]=stonk_currencies[stonk]


    for idx, row in singlestock_transactions.iterrows():

        multiplier = 1
        if(row['Action']=='buy'):
            multiplier=1
        elif(row['Action']=='sell'):
            multiplier=-1
        else:
            raise Exception("Transaction \"row['Action']\" is not recognized")

        #print(row['Amount'])


        #CHECK IF IT GETS APPLIED TO STARTING ARRAY, FIND AWAY TO DO THIS AT THE START
        #CONVERTING INTO AMOUNT MAKES it be adimensional, and doenst matter what currency it was bought in
        try:
            if(math.isnan(float(row['Amount']))):
                row['Amount']=row['Total Cost']/row['Cost per Share']
        except:
            row['Amount']=row['Total Cost']/row['Cost per Share']


        #df[row['When']:][stonk+" Amount"] = df[row['When']:][stonk+" Amount"]=row['Amount']

        #df[row['When']:,stonk+" Amount"] = df[row['When']:,stonk+" Amount"]+row['Amount']

        #print(float(row['Amount']))
        
        if pd.to_datetime("now")<pd.to_datetime(row['When']):
            print(row['When'],"<- This date is in the future, and will not be considere")
            continue


        
        #print(float(row['Amount'])*multiplier)
        #set amount to the amount
        #display(row['When'])
        #display(df.loc[row['When']:])
        
        
        
        
        #change the amount from whtn to forward,  multiplier determinex if its a  buy of a sell
        df.loc[row['When']:,'SharesOwned'] = df.loc[row['When']:,'SharesOwned'] + (float(row['Amount'])*multiplier)

        #money put in
        df.loc[row['When']:,'InvestedValue'] = df.loc[row['When']:,'InvestedValue'] + (float(row['Total Cost'])*multiplier)
        
        
        
        

        

    i=i+1
        
    df.loc[:,'TotalValue']= df["Price"]*df['SharesOwned']
    df.loc[:,'EarnedValue']= df.loc[:,'TotalValue']-df.loc[:,'InvestedValue']
    
    
        
    
    
    
    
    #delete rows with no data
    #
    df.interpolate(inplace=True)
    df.dropna(inplace=True,how='any')
    
    
    #display(df)
    #display(df)
    
    superdf= superdf.append(df)
    #display(df)
    #display(superdf)
    #print("YIKES")

    




True
XOM
WMT
WDC
V
11/02/2021 <- This date is in the future, and will not be considere
TSLA
TEAM
T
PG
PEP
ORCL
11/03/2021 <- This date is in the future, and will not be considere
NVDA
MSFT
MMM
11/02/2021 <- This date is in the future, and will not be considere
MCD
KO
INTC
IBM
GOOG
FB
DIS
CRM
BRK-B
BABA
AAL
AMZN
AMD
AAPL
DNNGY
05/01/2021 <- This date is in the future, and will not be considere
05/03/2021 <- This date is in the future, and will not be considere
JNJ
PLTR
ZM
12/03/2021 <- This date is in the future, and will not be considere
DIOD
NNND.F
IWDA.AS
12/02/2021 <- This date is in the future, and will not be considere
ETH-USD
11/02/2021 <- This date is in the future, and will not be considere
BTC-USD
11/02/2021 <- This date is in the future, and will not be considere


In [17]:

import random
import string

# printing lowercase
letters = string.ascii_lowercase

table = 'tampyy_'+''.join(random.choice(letters) for i in range(10)) 
print(table)

tampyy_yohrvwuwvn


In [18]:
#save to db historical data
dftodb  = superdf[['Price','Ticker']]
#connn = sqlite3.connect('example.db')
con.close()
con = sqlite3.connect('example.db')

dftodb.to_sql(table, con=con, if_exists='replace')


cur = con.cursor()

cur.execute("INSERT OR IGNORE INTO stocks_price SELECT * FROM {}".format(table))
cur.execute("DROP TABLE {}".format(table))
con.commit()
con.close()
#insert ticker by ticker, this way if there is a duplicate in db and it fails, it only fails for one single ticker
"""for ticker in dftodb['Ticker'].unique():
    print(ticker)
    
    try:        
        dftodb[dftodb['Ticker']==ticker].to_sql('stocks_price', con=connn, if_exists='append')
    except Exception as e:
        print(e)"""

"for ticker in dftodb['Ticker'].unique():\n    print(ticker)\n    \n    try:        \n        dftodb[dftodb['Ticker']==ticker].to_sql('stocks_price', con=connn, if_exists='append')\n    except Exception as e:\n        print(e)"

In [15]:
table

'gzlyyxvhkp'

In [ ]:
dftodb.loc[dftodb['Ticker']=='XOM','Ticker'] = 'HHHAA'

In [ ]:
dftodb
connn = sqlite3.connect('example.db')
dftodb.to_sql('stocks_price', con=connn, if_exists='append')

In [ ]:
superdf['CurrencyUSD']=0

In [ ]:
fauxpair = pairs.loc[:]['Close']

In [ ]:


hm = pd.merge(
    superdf,
    fauxpair,
    how="left",
    left_on='Date',
    right_on='Date',
    left_index=False,
    right_index=False,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

hm.loc[:]['Close']=hm.loc[:]['Close'].interpolate()




In [ ]:
import sqlite3
con = sqlite3.connect('example.db')

cur = con.cursor()

# Create table
cur.execute('''CREATE TABLE stocks_info (ticker text, info text,dateinserted TEXT )''')

               # Save (commit) the changes
#cur.execute('DROP TABLE stocks_info')
#cur.execute('DROP TABLE stocks')

con.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
con.close()

In [ ]:
from tinydb import TinyDB, Query
db = TinyDB('db.json')
db.insert({'type': 'apple', 'count': 7})

In [ ]:
!pip install tinydb

#df.loc[row['When']:,'Amount'] = df.loc[row['When']:,'Amount'] + (float(row['Amount'])*multiplier)
hm.loc[:,hm['Currency']=='EUR'] = hm.loc[:,'Close']*hm.loc[:,'TotalValue']

In [ ]:
#selecct column based on another column
hm.loc[hm['Currency']=='EUR',['Currency']]


#hm.loc[:,hm['Currency']=='EUR']['Close']
#hm[hm['Currency']=='EUR']['Currency']
#rectify us currency

#convert currency
hm.loc[:,'TotalValueUSD']=0
hm.loc[:,'TotalValueUSD']=hm.loc[:,'TotalValue']
hm.loc[hm['Currency']=='EUR','TotalValueUSD'] = hm[hm['Currency']=='EUR']['TotalValue']*hm[hm['Currency']=='EUR']['Close']

In [ ]:
print(time.time()-startu)

In [ ]:
#superdf[superdf['Ticker']=='XOM']
dfforaltair = superdf.loc["2019-10-20":].reset_index()
dfforaltair.interpolate(inplace=True)

alt.data_transformers.disable_max_rows()
alt.Chart(dfforaltair).mark_area().encode(
    x='Date',
    y='TotalValue',
    #color='Ticker:N',
    color=alt.Color('Ticker:N', legend=alt.Legend(columns=1,symbolLimit=300)),
).properties(
    width=1280,
    height=720
)

In [ ]:
for key in stonkdict:
    print(key)

702*0.50


df.loc['04/02/2020':, 'TSLAR'] = 2

df

In [ ]:
#df[df['1962-01-02':]][stonk+" Amount"]=df[df['1962-01-02':]][stonk+" Amount"]+row['Amount']



df

In [ ]:
!pip install --upgrade numpy
!pip install stocker

In [ ]:
SaveSummaryToDB(stock_summaries)